In [ ]:
import numpy as np
import scipy.io
from scipy.linalg import eigh
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# 1) Load QM7, extract 23 eigenvalues per Coulomb matrix => 23 features

def load_qm7_eigenvalues(matfile='qm7.mat'):
    """
    Loads QM7 from `matfile` and for each 23x23 Coulomb matrix
    computes the 23 sorted eigenvalues. Returns X_eig (N,23) and Y (N,).
    """
    data = scipy.io.loadmat(matfile)
    C = data['X']          # shape (N,23,23)
    Y = data['T'].ravel()  # shape (N,)
    N = C.shape[0]

    X_list = []
    for i in range(N):
        M = 0.5*(C[i] + C[i].T)    # symmetrize
        e_vals = eigh(M, eigvals_only=True)
        X_list.append(np.sort(e_vals))
    X_eig = np.array(X_list)      # (N,23)
    return X_eig, Y


# 2) Amplitude‐encoding helper 

def amplitude_encoding_circuit(vec, num_qubits):
    dim = 2**num_qubits
    qc = QuantumCircuit(num_qubits)
    padded = np.zeros(dim, dtype=complex)
    padded[:len(vec)] = vec
    norm = np.linalg.norm(padded)
    if norm < 1e-9:
        padded[0] = 1.0
        norm = 1.0
    padded /= norm
    qc.initialize(padded, range(num_qubits))
    return qc

def build_statevectors(X, num_qubits):
    N = X.shape[0]
    dim = 2**num_qubits
    sv = np.zeros((N, dim), dtype=complex)
    for i in range(N):
        qc = amplitude_encoding_circuit(X[i], num_qubits)
        sv[i] = Statevector.from_instruction(qc).data
    return sv

def compute_kernel_statevector(sv):
    N = sv.shape[0]
    K = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            ov = np.vdot(sv[i], sv[j])
            K[i,j] = abs(ov)**2
            K[j,i] = K[i,j]
    return K


# 3) Main: QSVR on 23-dimensional eigenvalues

def main_qsvr_on_eigenvalues(
    matfile='qm7.mat',
    subset_size=600,
    test_size=0.2,
    num_qubits=5,       # minimum 5 to encode 23 dims
    random_seed=42
):
    """
    1) Loads dataset, flatten => 529 features
    2) Subsamples
    3) Scales features with StandardScaler
    4) Scales targets (subtract mean, divide std)
    5) Train/test split
    6) Amplitude encode -> direct statevector kernel
    7) Fits SVR
    8) inverts target transform, measures final MAE, R^2
    """

    # 1) Loads eigenvalues
    X_all, Y_all = load_qm7_eigenvalues(matfile)
    Ntotal = len(X_all)
    print(f"Loaded {Ntotal} molecules with 23 eigenvalue features.")

    # 2) Subsamples
    np.random.seed(random_seed)
    idxs = np.random.choice(Ntotal, subset_size, replace=False)
    X_sub, Y_sub = X_all[idxs], Y_all[idxs]

    # 3) Scales features & targets
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_sub)
    Y_mean, Y_std = Y_sub.mean(), Y_sub.std()
    Y_scaled = (Y_sub - Y_mean)/Y_std

    # 4) Train/test split
    X_train, X_test, Y_train, Y_test = train_test_split(
        X_scaled, Y_scaled,
        test_size=test_size, random_state=random_seed
    )
    print(f"Train={X_train.shape[0]}, Test={X_test.shape[0]}. Using {num_qubits} qubits.")

    # 5) Builds kernels via amplitude‐encoded statevectors
    print("Building train statevectors...")
    sv_train = build_statevectors(X_train, num_qubits)
    print("Computing train kernel...")
    K_train = compute_kernel_statevector(sv_train)

    # 6) Fits SVR (precomputed)
    svr = SVR(kernel='precomputed', C=10.0, epsilon=1.0)
    svr.fit(K_train, Y_train)

    # 7) Tests kernel
    print("Building test statevectors...")
    sv_test = build_statevectors(X_test, num_qubits)
    K_test = np.array([[abs(np.vdot(sv_test[i], sv_train[j]))**2
                        for j in range(len(sv_train))]
                       for i in range(len(sv_test))])

    # 8) Predicts & inverts scaling
    Y_pred_scaled = svr.predict(K_test)
    Y_pred = Y_pred_scaled * Y_std + Y_mean
    Y_true = Y_test  * Y_std + Y_mean

    # 9) Evaluates
    print("\nQSVR on 23-D eigenvalues (amp-encoded):")
    print(f" MAE: {mean_absolute_error(Y_true, Y_pred):.3f}")
    print(f"  R2: {r2_score(Y_true, Y_pred):.3f}")

if __name__ == "__main__":
    main_qsvr_on_eigenvalues(
        matfile='qm7.mat',
        subset_size=1000,
        test_size=0.2,
        num_qubits=5,
        random_seed=42
    )
